In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
%pip install albumentations==1.3.0
import albumentations as A
from albumentations.pytorch import ToTensorV2

train_transforms = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=45, p=0.5),
    A.CoarseDropout(max_holes=1, max_height=16, max_width=16, min_holes=1, min_height=16, min_width=16, fill_value=(0.4914, 0.4822, 0.4465), mask_fill_value=None, p=0.5),  # Updated fill_value
    A.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2470, 0.2435, 0.2616)),
    ToTensorV2(),
])

test_transforms = A.Compose([
    A.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2470, 0.2435, 0.2616)),
    ToTensorV2(),
])

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.4.20
    Uninstalling albumentations-1.4.20:
      Successfully uninstalled albumentations-1.4.20


In [3]:
import numpy as np
from torchvision.datasets import CIFAR10

class AlbumentationsDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):

        image, label = self.dataset[idx]
        image = np.array(image)
        # Apply albumentations transforms
        if self.transform:
            image = self.transform(image=image)["image"]
        return image, label

# Load CIFAR-10 dataset without transforms (albumentations will handle it)
train_dataset = CIFAR10(root='./data', train=True, download=True, transform=None)
test_dataset = CIFAR10(root='./data', train=False, download=True, transform=None)

# Wrap the datasets with AlbumentationsDataset
train_dataset = AlbumentationsDataset(train_dataset, transform=train_transforms)
test_dataset = AlbumentationsDataset(test_dataset, transform=test_transforms)


100%|██████████| 170M/170M [00:02<00:00, 78.4MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train_dataset, **dataloader_args) # Changed 'train' to 'train_dataset'

# test dataloader
test_loader = torch.utils.data.DataLoader(test_dataset, **dataloader_args) # Changed 'test' to 'test_dataset'

CUDA Available? False


In [5]:
#code 2
import torch.nn as nn
import torch.nn.functional as F

class CIFAR10(nn.Module):
    def __init__(self):
        super(CIFAR10, self).__init__()

        # C1:
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=8, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.Dropout(0.1),

            nn.Conv2d(in_channels=8, out_channels=10, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(10),
            nn.ReLU(),
            nn.Dropout(0.1),

            nn.Conv2d(in_channels=10, out_channels=20, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(20),
            nn.ReLU(),
            nn.Dropout(0.1),

            nn.Conv2d(in_channels=20, out_channels=32, kernel_size=(3,3),stride=2,dilation=2,padding=1),  #Downsampling
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(0.1),

        )

        # C2:
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=48, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(48),
            nn.ReLU(),
            nn.Dropout(0.1),

            nn.Conv2d(in_channels=48, out_channels=64, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(0.1),

            nn.Conv2d(in_channels=64, out_channels=48, kernel_size=(3, 3), dilation=2, stride =2, padding=2),  # Dilated convolution, Downsampling
            nn.BatchNorm2d(48),
            nn.ReLU(),
            nn.Dropout(0.1),


        )

        # C3:
        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=48, out_channels=32, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(0.1),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3),padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(0.1),

            nn.Conv2d(in_channels=64, out_channels=16, kernel_size=(3, 3), dilation=2, padding=2,stride=2),  # Dilated convolution, Downsampling
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Dropout(0.1),

        )

        # C4:
        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(0.05),

            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(3, 3), padding=1,dilation=2), #added dilation
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(0.05),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3), stride=1, padding=1,groups=32), #Depthwise convolution
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(1, 1)),  # Pointwise convolution
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(0.05),

        )

        # OUTPUT: Global Average Pooling
        self.gap = nn.Sequential(
            nn.AdaptiveAvgPool2d((1,1))  # Adjust kernel size based on input dimensions
        )

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.convblock4(x)
        x = self.gap(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor to [batch_size, 10]
        return F.log_softmax(x, dim=1)


In [7]:
%pip install torchsummary
from torchsummary import summary

model = CIFAR10()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
summary(model, input_size=(3, 32, 32))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 32, 32]             224
       BatchNorm2d-2            [-1, 8, 32, 32]              16
              ReLU-3            [-1, 8, 32, 32]               0
           Dropout-4            [-1, 8, 32, 32]               0
            Conv2d-5           [-1, 10, 32, 32]             730
       BatchNorm2d-6           [-1, 10, 32, 32]              20
              ReLU-7           [-1, 10, 32, 32]               0
           Dropout-8           [-1, 10, 32, 32]               0
            Conv2d-9           [-1, 20, 32, 32]           1,820
      BatchNorm2d-10           [-1, 20, 32, 32]              40
             ReLU-11           [-1, 20, 32, 32]               0
          Dropout-12           [-1, 20, 32, 32]               0
           Conv2d-13           [-1, 32, 15, 15]           5,792
      BatchNorm2d-14           [-1, 32,

In [ ]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm

    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    test_acc.append(100. * correct / len(test_loader.dataset))
    return test_loss,test_acc

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.optim as optim

model = CIFAR10().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.005, weight_decay=1e-4)


scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.75, patience=3, verbose=True)

EPOCHS = 150

for epoch in range(1, EPOCHS + 1):
    print("EPOCH:", epoch)

    # Train the model
    train(model, device, train_loader, optimizer, epoch)

    test_loss, test_acc = test(model, device, test_loader)
    scheduler.step(test_loss)


EPOCH: 1


Loss=1.784204125404358 Batch_id=390 Accuracy=30.38: 100%|██████████| 391/391 [00:22<00:00, 17.16it/s]



Test set: Average loss: 1.8098, Accuracy: 4465/10000 (44.65%)

EPOCH: 2


Loss=1.9006723165512085 Batch_id=390 Accuracy=38.89: 100%|██████████| 391/391 [00:20<00:00, 19.02it/s]



Test set: Average loss: 1.7824, Accuracy: 4194/10000 (41.94%)

EPOCH: 3


...


Test set: Average loss: 0.4474, Accuracy: 8531/10000 (85.31%)

EPOCH: 149


Loss=0.954598069190979 Batch_id=390 Accuracy=71.56: 100%|██████████| 391/391 [00:22<00:00, 17.28it/s]



Test set: Average loss: 0.4472, Accuracy: 8524/10000 (85.24%)

EPOCH: 150


Loss=0.8123047947883606 Batch_id=390 Accuracy=72.11: 100%|██████████| 391/391 [00:22<00:00, 17.73it/s]



Test set: Average loss: 0.4487, Accuracy: 8535/10000 (85.35%)

